# 1. Load & Inspect Dataset

In [9]:
# Import Libraries
import numpy as np
import pandas as pd
from shapely import wkt

import warnings
warnings.filterwarnings('ignore')

In [10]:
# load the dataset
df= pd.read_csv(r"D:\#Great Learning\Capstone Project\Datasets\Traffic_Crash.csv")

In [11]:
df.head()
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                300000 non-null  object 
 1   CRASH_DATE_EST_I               21757 non-null   object 
 2   CRASH_DATE                     300000 non-null  object 
 3   POSTED_SPEED_LIMIT             300000 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         300000 non-null  object 
 5   DEVICE_CONDITION               300000 non-null  object 
 6   WEATHER_CONDITION              300000 non-null  object 
 7   LIGHTING_CONDITION             300000 non-null  object 
 8   FIRST_CRASH_TYPE               300000 non-null  object 
 9   TRAFFICWAY_TYPE                300000 non-null  object 
 10  LANE_CNT                       59529 non-null   float64
 11  ALIGNMENT                      300000 non-null  object 
 12  ROADWAY_SURFACE_COND          

,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
count,300000,21757,300000,300000.00000,300000,300000,300000,300000,300000,300000,...,299355.000000,299355.000000,299355.000000,299355.0,300000.000000,300000.000000,300000.000000,297699.000000,297699.000000,297699
unique,300000,2,254547,NaN,19,8,12,6,18,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152791
top,ee0b70ab08d3cbd410c1f3ecb3c3516f8038fe8047a0d6...,Y,02-06-2025 08:00,NaN,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.905309125103 41.976201139024)
freq,1,18969,10,NaN,169185,171278,236215,192890,69363,128524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,447
mean,NaN,NaN,NaN,28.42684,NaN,NaN,NaN,NaN,NaN,NaN,...,0.110100,0.067789,1.999155,0.0,13.180733,4.117217,6.686750,41.856194,-87.675518,NaN
std,NaN,NaN,NaN,6.05171,NaN,NaN,NaN,NaN,NaN,NaN,...,0.425147,0.336534,1.169587,0.0,5.574193,1.981695,3.363232,0.279365,0.559775,NaN
min,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,-87.934763,NaN
25%,NaN,NaN,NaN,30.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.000000,0.0,9.000000,2.000000,4.000000,41.783866,-87.721945,NaN
50%,NaN,NaN,NaN,30.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,2.000000,0.0,14.000000,4.000000,7.000000,41.875131,-87.674501,NaN
75%,NaN,NaN,NaN,30.00000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,2.000000,0.0,17.000000,6.000000,10.000000,41.924678,-87.633779,NaN


# Standardizing Column Names

In [12]:
# # Convert column names to lowercase + snake_case
# df.columns= (
#     df.columns.str.strip()
#               .str.lower()
#               .str.replace(' ','_')
#               .str.replace('(','')
#               .str.replace(')','')
#               .str.replace('-','_')
# )
# df.head()

# Handle Date & Time Columns

In [13]:
# dataset has CRASH_DATE and DATE_POLICE_NOTIFIED in string formats:
# convert them into date time

df['CRASH_DATE'] = pd.to_datetime(df['CRASH_DATE'], errors='coerce')
df['DATE_POLICE_NOTIFIED'] = pd.to_datetime(df['DATE_POLICE_NOTIFIED'], errors='coerce')


In [14]:
#Create useful new features.
df['CRASH_YEAR'] = df['CRASH_DATE'].dt.year
df['CRASH_MONTH'] = df['CRASH_DATE'].dt.month
df['CRASH_HOUR'] = df['CRASH_DATE'].dt.hour
df['CRASH_DAY'] = df['CRASH_DATE'].dt.day_name()


# Clean Location Columns

In [15]:
# LATITUDE, LONGITUDE, LOCATION must be separated. If LOCATION contains (lat lon), extract clean numeric values:
df['LATITUDE'] = pd.to_numeric(df['LATITUDE'], errors='coerce')
df['LONGITUDE'] = pd.to_numeric(df['LONGITUDE'], errors='coerce')


In [16]:
# Drop LOCATION column (redundant):
df.drop(columns=['LOCATION'], inplace=True)


# Standardize Boolean Columns

In [17]:
# Columns such as INTERSECTION_RELATED_I, HIT_AND_RUN_I, WORK_ZONE_I, PHOTOS_TAKEN_I
# contain values like Y / N or blank.

# Convert to binary format:
columns = ['INTERSECTION_RELATED_I','NOT_RIGHT_OF_WAY_I','HIT_AND_RUN_I','PHOTOS_TAKEN_I',
           'STATEMENTS_TAKEN_I','DOORING_I','WORK_ZONE_I','WORKERS_PRESENT_I']

for col in columns:
    df[col] = df[col].map({'Y':1,'N':0})
    df[col] = df[col].fillna(0)


# Fix Injury Columns

In [18]:
#Dataset has many injury-related columns like:
# INJURIES_TOTAL, INJURIES_FATAL, INJURIES_REPORTED_NOT_EVIDENT, ...

# Convert them into integer type & fill NAs:

inj_cols = [col for col in df.columns if 'INJURIES' in col]

for col in inj_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)


# Handle Categorical Columns

In [23]:
# We have many categorical fields (WEATHER_CONDITION, FIRST_CRASH_TYPE, etc.)

# Encode using LabelEncoder or One-Hot Encoding
# cat_cols = df.select_dtypes(include='object').columns
# df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

from sklearn.preprocessing import LabelEncoder

cat_cols = df.select_dtypes(include='object').columns
le = LabelEncoder()

for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))



# Handle Missing Values

In [21]:
# For numeric columns — use median
# For categorical columns — use mode

num = df.select_dtypes(include='number').columns
cat = df.select_dtypes(include='object').columns

df[num] = df[num].fillna(df[num].median())
df[cat] = df[cat].fillna(df[cat].mode().iloc[0])


In [22]:
# Remove Irrelevant Columns
# Drop IDs, exact addresses if not needed for prediction
df.drop(columns=['CRASH_RECORD_ID'], inplace=True)

# If street name & number not useful:
df.drop(columns=['STREET_NO','STREET_NAME','STREET_DIRECTION'], inplace=True)


In [24]:
df.head()

,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CRASH_YEAR,CRASH_DAY
0,1,2024-06-23 14:27:00,30,16,1,2,3,17,11,2.0,...,0,2,0,14,1,6,41.864897,-87.616781,2024,3
1,1,2025-02-22 03:00:00,30,4,3,2,1,7,10,2.0,...,0,1,0,3,7,2,41.782577,-87.614322,2025,2
2,1,2021-10-27 14:16:00,30,4,3,2,3,14,8,2.0,...,0,3,0,14,4,10,41.845306,-87.644169,2021,6
3,1,2023-07-05 17:00:00,30,16,6,2,3,15,2,2.0,...,0,2,0,17,4,7,41.953968,-87.727479,2023,6
4,1,2019-09-11 08:30:00,30,15,1,2,3,10,2,2.0,...,0,2,0,8,4,9,41.930973,-87.779905,2019,6


# Final Clean Dataset

In [25]:
df.to_csv(r"D:\#Great Learning\Capstone Project\clean_data\cleaned_crash_data.csv", index=False)
